In [87]:
import h5py
from util import *

In [109]:
old_data = load_dict_from_hdf5("../../STARmap_2018/previous_runs/old_format/NMF_K15_rs0.h5")
print(old_data["hyperparameters"].keys())

dict_keys(['K', 'betas', 'lambda_SigmaXInv', 'lambda_x', 'prior_x_modes', 'repli_list'])


In [89]:
with h5py.File("../../data/synthetic_metagene_free_cortex/500_cells_100_genes_separate_low_variance_low_covariance/results_with_spatial_low_lambda.hdf5", 'r') as h5file:
    print(h5file.keys())

<KeysViewHDF5 ['dataset', 'hyperparameters', 'parameters', 'progress', 'weights']>


In [90]:
dataset = load_dict_from_hdf5("../../data/synthetic_metagene_free_cortex/500_cells_100_genes_separate_low_variance_low_covariance/results_with_spatial_low_lambda.hdf5")

In [91]:
dataset["parameters"].keys()

dict_keys(['M', 'prior_x_parameter', 'sigma_x_inverse', 'sigma_yx_inverses'])

In [110]:
def reformat_result(old_filepath, sister_filepath, new_filepath):
    """Reformat SpiceMix result to be compatible with new Result implementation.
    
    Args:
        old_filepath: path to SpiceMix result in old format
        sister_filepath: path to SpiceMix result from same dataset in new path
        new_filepath: output filepath for new SpiceMix result
    """
    
    old_data = load_dict_from_hdf5(old_filepath)
    dataset = load_dict_from_hdf5(sister_filepath)["dataset"]
    new_data = {}
    
    hyperparameters = old_data["hyperparameters"]
    repli_list = [repli.decode("utf-8") for repli in hyperparameters["repli_list"]]

    replicate_mapping = {
        replicate_name: str(replicate_index) for replicate_index, replicate_name in enumerate(repli_list)
    }
    
    del hyperparameters["repli_list"]
    
    def remap(dictionary, mapping):
        new_dictionary = {}
        for key in dictionary:
            new_key = mapping[key]
            new_dictionary[new_key] = dictionary[key]
            
        return new_dictionary
    
    hyperparameters["lambda_sigma_x_inverse"] = hyperparameters["lambda_SigmaXInv"]
    hyperparameters["prior_x_modes"] = remap(hyperparameters["prior_x_modes"], replicate_mapping)
    del hyperparameters["lambda_SigmaXInv"]

    parameters = old_data["parameters"]
    parameters["sigma_x_inverse"] = old_data["parameters"]["Sigma_x_inv"]
    parameters["prior_x_parameter"] = remap(old_data["parameters"]["prior_xs"], replicate_mapping)
    parameters["sigma_yx_inverses"] = remap(old_data["parameters"]["sigma_yx_invs"], replicate_mapping)
    del parameters["Sigma_x_inv"]
    del parameters["prior_xs"]
    del parameters["sigma_yx_invs"]

    new_data["hyperparameters"] = hyperparameters
    new_data["weights"] = remap(old_data["latent_states"]["XT"], replicate_mapping)
    new_data["progress"] = old_data["progress"]
    new_data["dataset"] = dataset
    new_data["parameters"] = parameters
    
    save_dict_to_hdf5(new_filepath, new_data)

In [111]:
# Reform SeqFISHPlus runs
sister_filepath = "../../SeqFISHPlus/results_with_spatial_low_lambda.hdf5"
for method in ["NMF_K20", "SpiceMix_K20"]:
    for random_seed in range(5):
        old_filepath = f"../../SeqFISHPlus/previous_runs/old_format/{method}_niter500_rs{random_seed}.h5"
        new_filepath = f"../../SeqFISHPlus/previous_runs/new_format/{method}_random_seed_{random_seed}.hdf5"
        reformat_result(old_filepath, sister_filepath, new_filepath)

In [112]:
# Reform STARmap_2018 runs
sister_filepath = "../../STARmap_2018/results_with_spatial.hdf5"
for method in ["NMF_K15", "SpiceMix_K20"]:
    for random_seed in range(5):
        old_filepath = f"../../STARmap_2018/previous_runs/old_format/{method}_rs{random_seed}.h5"
        new_filepath = f"../../STARmap_2018/previous_runs/new_format/{method}_random_seed_{random_seed}.hdf5"
        reformat_result(old_filepath, sister_filepath, new_filepath)

In [76]:
new_data = load_dict_from_hdf5("../../SeqFISHPlus/previous_runs/new_format/SpiceMix_K20_random_seed_0.hdf5")

In [80]:
new_data["hyperparameters"]

{'K': 20,
 'betas': array([0.2, 0.2, 0.2, 0.2, 0.2]),
 'lambda_sigma_x_inverse': 0.0001,
 'prior_x_modes': {'0': b'Exponential shared fixed',
  '1': b'Exponential shared fixed',
  '2': b'Exponential shared fixed',
  '3': b'Exponential shared fixed',
  '4': b'Exponential shared fixed'}}